In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import monotonically_increasing_id, col, collect_list, concat_ws, udf, expr, split
from pyspark.sql.types import IntegerType, ArrayType, StringType, DoubleType, MapType
import pyspark.sql.functions as f
from pyspark.sql import Window

from pyspark.ml.feature import Tokenizer as MLTokenizer
from pyspark.ml.feature import HashingTF, IDF, CountVectorizer, StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, VectorUDT

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder \
    .appName("SparkNLP-TFIDF")\
    .config("spark.executor.instances", "2") \
    .config("spark.executor.memory", "30G") \
    .config("spark.executor.cores", "24") \
    .config("spark.executor.cores.max", "24") \
    .config("spark.driver.memory", "20G") \
  
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.4")\
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jpietraszek/.ivy2/cache
The jars for the packages stored in: /home/jpietraszek/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8caaba90-4029-459a-8c77-c282e561b048;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.4 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.gu

In [3]:
%%time
df = spark.read.json('/user/jpietraszek/arxiv-metadata-oai-snapshot.json')

CPU times: user 12.1 ms, sys: 2.36 ms, total: 14.4 ms
Wall time: 10.5 s


In [4]:
%%time
df = df.filter(~col("categories").like("% %"))
df = df.withColumn("abstract", f.lower(col("abstract")))
df = df.withColumn("abstract", f.regexp_replace(col("abstract"), "[^a-zA-Z\\s]", ""))


CPU times: user 4.43 ms, sys: 2.44 ms, total: 6.87 ms
Wall time: 117 ms


In [5]:
#df = df.sample(fraction = .001)

In [10]:
%%time
documentAssembler = DocumentAssembler() \
.setInputCol("abstract") \
.setOutputCol("document")

sentenceDetector = SentenceDetector() \
.setInputCols(["document"]) \
.setOutputCol("sentence")

tokenizer = Tokenizer() \
.setInputCols(["sentence"]) \
.setOutputCol("token")

stemmer = Stemmer() \
.setInputCols(["token"]). \
setOutputCol("stem")

lemmatizer = LemmatizerModel.pretrained("lemma_spacylookup","en") \
.setInputCols(["stem"]) \
.setOutputCol("lemma")

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["lemma"]) \
    .setOutputCol("clean_lemma") \
    .setCaseSensitive(False)

pipeline = Pipeline() \
.setStages([
documentAssembler,
sentenceDetector,
tokenizer,
stemmer,
lemmatizer,
stopwords_cleaner
])

lemma_spacylookup download started this may take some time.
Approximate size to download 417 KB
[OK!]
CPU times: user 27 ms, sys: 11.4 ms, total: 38.4 ms
Wall time: 4.57 s


In [11]:
%%time
result = pipeline.fit(df).transform(df)

CPU times: user 41.7 ms, sys: 8.13 ms, total: 49.9 ms
Wall time: 335 ms


In [12]:
df.select('abstract').first()

Row(abstract='  a fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders all\nnexttoleading order perturbative contributions from quarkantiquark\ngluonantiquark and gluongluon subprocesses are included as well as\nallorders resummation of initialstate gluon radiation valid at\nnexttonexttoleading logarithmic accuracy the region of phase space is\nspecified in which the calculation is most reliable good agreement is\ndemonstrated with data from the fermilab tevatron and predictions are made for\nmore detailed tests with cdf and do data predictions are shown for\ndistributions of diphoton pairs produced at the energy of the large hadron\ncollider lhc distributions of the diphoton pairs from the decay of a higgs\nboson are contrasted with those produced from qcd processes at the lhc showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events\n')

In [13]:
result.selectExpr("clean_lemma.result").first()

Row(result=['fulli', 'differenti', 'calcul', 'perturb', 'quantum', 'chromodynam', 'present', 'product', 'massiv', 'photon', 'pair', 'hadron', 'collid', 'nexttolead', 'order', 'perturb', 'contribut', 'quarkantiquark', 'gluonantiquark', 'gluongluon', 'subprocess', 'ar', 'includ', 'good', 'allord', 'resumm', 'initialst', 'gluon', 'radiat', 'valid', 'nexttonexttolead', 'logarithm', 'accuraci', 'region', 'phase', 'space', 'specifi', 'calcul', 'much', 'reliabl', 'good', 'agreem', 'demonstr', 'datum', 'fermilab', 'tevatron', 'predict', 'ar', 'make', 'much', 'detail', 'test', 'cdf', 'datum', 'predict', 'ar', 'show', 'distribut', 'diphoton', 'pair', 'produc', 'energi', 'larg', 'hadron', 'collid', 'lhc', 'distribut', 'diphoton', 'pair', 'decai', 'higg', 'boson', 'ar', 'contrast', 'produc', 'qcd', 'process', 'lhc', 'showe', 'enhanc', 'sensit', 'signal', 'obtain', 'judici', 'select', 'event'])

In [12]:
%%time
lemma_df = result.selectExpr("id","clean_lemma.result","categories")
lemma_df.write.format('parquet').mode("overwrite").save("/user/jpietraszek/arxiv-lemmatized_preprocessed_spacy.parquet")

CPU times: user 97.6 ms, sys: 27.6 ms, total: 125 ms
Wall time: 2min 32s


In [9]:
%%time 
tmp = spark.read.parquet('/user/jpietraszek/arxiv-lemmatized_preprocessed.parquet')

CPU times: user 1.7 ms, sys: 1.59 ms, total: 3.29 ms
Wall time: 211 ms


23/11/18 12:48:54 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
23/11/18 12:48:54 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [24]:
%%time 
tmp = spark.read.parquet('/user/jpietraszek/arxiv-lemmatized_preprocessed.parquet')

CPU times: user 1.72 ms, sys: 1.77 ms, total: 3.49 ms
Wall time: 123 ms


In [10]:
merged_df = merged_df.groupBy("categories").agg(concat_ws(" ", collect_list("result")).alias("merged_result"))

merged_df = merged_df.withColumn("text_length", length_udf(col("merged_result")))
merged_df = merged_df.orderBy(col("text_length").desc())

longest_text_row = merged_df.first()

In [11]:
merged_df.show()

+-----------------+--------------------+-----------+
|       categories|       merged_result|text_length|
+-----------------+--------------------+-----------+
|         astro-ph|Massive Black Hol...|      85308|
|         quant-ph|The strong light-...|      59939|
|           hep-ph|In QCD with $N_c$...|      54516|
|            cs.CV|Open-set logo rec...|      42189|
|      astro-ph.GA|The L204 dark clo...|      36410|
|           hep-th|The renormalized ...|      36009|
|      astro-ph.SR|Hot subdwarf B st...|      35491|
|cond-mat.mtrl-sci|The existence of ...|      35320|
|      astro-ph.HE|We present a stat...|      31747|
|            gr-qc|The crucial role ...|      29877|
|cond-mat.mes-hall|The quasiparticle...|      28732|
|          math.AP|We consider a con...|      24154|
|    cs.IT math.IT|We examine the pr...|      23945|
|          math.CO|What be the unavo...|      23486|
|    cs.LG stat.ML|Differential priv...|      22338|
|            cs.CL|This paper descri...|      

In [12]:
merged_df = merged_df.withColumn("TextArray", split("merged_result", " "))

In [13]:
merged_df.show()

+-----------------+--------------------+-----------+--------------------+
|       categories|       merged_result|text_length|           TextArray|
+-----------------+--------------------+-----------+--------------------+
|         astro-ph|Massive Black Hol...|      85308|[Massive, Black, ...|
|         quant-ph|The strong light-...|      59939|[The, strong, lig...|
|           hep-ph|In QCD with $N_c$...|      54516|[In, QCD, with, $...|
|            cs.CV|Open-set logo rec...|      42189|[Open-set, logo, ...|
|      astro-ph.GA|The L204 dark clo...|      36410|[The, L204, dark,...|
|           hep-th|The renormalized ...|      36009|[The, renormalize...|
|      astro-ph.SR|Hot subdwarf B st...|      35491|[Hot, subdwarf, B...|
|cond-mat.mtrl-sci|The existence of ...|      35320|[The, existence, ...|
|      astro-ph.HE|We present a stat...|      31747|[We, present, a, ...|
|            gr-qc|The crucial role ...|      29877|[The, crucial, ro...|
|cond-mat.mes-hall|The quasiparticle..

In [73]:
from sparknlp.base import DocumentAssembler, Pipeline, LightPipeline
from sparknlp.annotator import (
    SentenceDetector,
    Tokenizer,
    YakeKeywordExtraction
)
import pyspark.sql.functions as F

# Step 1: Transforms raw texts to `document` annotation
document = DocumentAssembler() \
            .setInputCol("text") \
            .setOutputCol("document")

# Step 2: Sentence Detection
sentenceDetector = SentenceDetector() \
            .setInputCols("document") \
            .setOutputCol("sentence")

# Step 3: Tokenization
token = Tokenizer() \
            .setInputCols("sentence") \
            .setOutputCol("token") \
            .setContextChars(["(", ")", "?", "!", ".", ","])

# Step 4: Keyword Extraction
keywords = YakeKeywordExtraction() \
            .setInputCols("token") \
            .setOutputCol("keywords") \
            
# Define the pipeline
yake_pipeline = Pipeline(stages=[document, sentenceDetector, token, keywords])

# Create an empty dataframe
empty_df = spark.createDataFrame([['']]).toDF("text")

# Fit the dataframe to get the 
yake_Model = yake_pipeline.fit(empty_df)

In [74]:
light_model = LightPipeline(yake_Model)

text = '''
google is acquiring data science community kaggle. Sources tell us that google is acquiring kaggle, a platform that hosts data science and machine learning competitions. Details about the transaction remain somewhat vague , but given that google is hosting its Cloud Next conference in san francisco this week, the official announcement could come as early as tomorrow. Reached by phone, kaggle co-founder ceo anthony goldbloom declined to deny that the acquisition is happening. google itself declined 'to comment on rumors'. kaggle, which has about half a million data scientists on its platform, was founded by Goldbloom and Ben Hamner in 2010. The service got an early start and even though it has a few competitors like DrivenData, TopCoder and HackerRank, it has managed to stay well ahead of them by focusing on its specific niche. The service is basically the de facto home for running data science and machine learning competitions. With kaggle, google is buying one of the largest and most active communities for data scientists - and with that, it will get increased mindshare in this community, too (though it already has plenty of that thanks to Tensorflow and other projects). kaggle has a bit of a history with google, too, but that's pretty recent. Earlier this month, google and kaggle teamed up to host a $100,000 machine learning competition around classifying YouTube videos. That competition had some deep integrations with the google Cloud platform, too. Our understanding is that google will keep the service running - likely under its current name. While the acquisition is probably more about Kaggle's community than technology, kaggle did build some interesting tools for hosting its competition and 'kernels', too. On kaggle, kernels are basically the source code for analyzing data sets and developers can share this code on the platform (the company previously called them 'scripts'). Like similar competition-centric sites, kaggle also runs a job board, too. It's unclear what google will do with that part of the service. According to Crunchbase, kaggle raised $12.5 million (though PitchBook says it's $12.75) since its launch in 2010. Investors in kaggle include Index Ventures, SV Angel, Max Levchin, Naval Ravikant, google chief economist Hal Varian, Khosla Ventures and Yuri Milner
'''

light_result = light_model.fullAnnotate(text)[0]

In [75]:
light_result

{'document': [Annotation(document, 0, 2318, 
  google is acquiring data science community kaggle. Sources tell us that google is acquiring kaggle, a platform that hosts data science and machine learning competitions. Details about the transaction remain somewhat vague , but given that google is hosting its Cloud Next conference in san francisco this week, the official announcement could come as early as tomorrow. Reached by phone, kaggle co-founder ceo anthony goldbloom declined to deny that the acquisition is happening. google itself declined 'to comment on rumors'. kaggle, which has about half a million data scientists on its platform, was founded by Goldbloom and Ben Hamner in 2010. The service got an early start and even though it has a few competitors like DrivenData, TopCoder and HackerRank, it has managed to stay well ahead of them by focusing on its specific niche. The service is basically the de facto home for running data science and machine learning competitions. With kaggle